# NEXO to BTC Exchange Opportunity Monitor

This notebook monitors the NEXO/BTC price ratio and identifies favorable moments to exchange NEXO for Bitcoin based on local peaks in the ratio.

## Setup and Dependencies

First, let's install the required packages:

In [ ]:
!pip install ccxt pandas numpy matplotlib

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import logging
from typing import Dict, List, Tuple, Optional
import os
import sys
from IPython.display import display, HTML
from google.colab import files

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger("NexoBTCExchange")

## Helper Functions

Define helper functions for calculating percentiles:

In [ ]:
def percentileofscore(a, score):
    """
    Calculate the percentile rank of a score relative to a list of scores.
    
    This function is a simplified version of scipy.stats.percentileofscore
    for use when scipy is not available.
    
    Args:
        a: Array of scores to find the percentile in
        score: Score to find the percentile of
        
    Returns:
        The percentile rank (0-100) of the score
    """
    a = np.asarray(a)
    n = len(a)
    
    if n == 0:
        return np.nan
        
    # Number of scores below score
    below = np.sum(a < score)
    # Number of scores equal to score
    equal = np.sum(a == score)
    
    # Linear interpolation between points
    if equal:
        return 100.0 * (below + 0.5 * equal) / n
    else:
        return 100.0 * below / n

## Main Monitor Class

Now let's define the main class that will monitor the exchange rates:

In [ ]:
class NexoBTCExchangeMonitor:
    """Monitor Nexo/BTC price ratio and identify exchange opportunities."""
    
    def __init__(
        self,
        exchange_id: str = 'binance',
        nexo_symbol: str = 'NEXO/USDT',
        btc_symbol: str = 'BTC/USDT',
        timeframe: str = '1h',
        window_size: int = 168,  # 1 week worth of hourly data
        peak_threshold: float = 0.9,  # 90th percentile
        data_dir: str = './data'
    ):
        """
        Initialize the exchange monitor.
        
        Args:
            exchange_id: CCXT exchange ID to use
            nexo_symbol: Nexo trading pair symbol
            btc_symbol: Bitcoin trading pair symbol
            timeframe: Timeframe for historical data
            window_size: Number of periods to analyze for trends
            peak_threshold: Percentile threshold to identify peaks (0.0-1.0)
            data_dir: Directory to save data
        """
        self.exchange_id = exchange_id
        self.nexo_symbol = nexo_symbol
        self.btc_symbol = btc_symbol
        self.timeframe = timeframe
        self.window_size = window_size
        self.peak_threshold = peak_threshold
        self.data_dir = data_dir
        
        # Create data directory if it doesn't exist
        os.makedirs(data_dir, exist_ok=True)
        
        # Initialize exchange
        self.exchange = self._init_exchange()
        
        # Initialize historical data storage
        self.history_df = self._init_history_df()
        
    def _init_exchange(self) -> ccxt.Exchange:
        """Initialize and return CCXT exchange instance."""
        try:
            exchange_class = getattr(ccxt, self.exchange_id)
            exchange = exchange_class({
                'enableRateLimit': True,
            })
            logger.info(f"Successfully initialized {self.exchange_id} exchange")
            return exchange
        except Exception as e:
            logger.error(f"Failed to initialize exchange: {e}")
            raise
    
    def _init_history_df(self) -> pd.DataFrame:
        """Initialize or load historical data DataFrame."""
        history_file = os.path.join(self.data_dir, f"nexo_btc_history_{self.timeframe}.csv")
        
        if os.path.exists(history_file):
            try:
                df = pd.read_csv(history_file)
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                logger.info(f"Loaded historical data from {history_file}")
                return df
            except Exception as e:
                logger.warning(f"Failed to load historical data: {e}. Creating new dataset.")
        
        # Create empty DataFrame
        return pd.DataFrame(columns=[
            'timestamp', 'nexo_price', 'btc_price', 'nexo_btc_ratio',
            'ratio_sma', 'ratio_percentile', 'is_peak'
        ])
    
    def fetch_current_prices(self) -> Tuple[float, float]:
        """Fetch current prices for Nexo and BTC."""
        try:
            nexo_ticker = self.exchange.fetch_ticker(self.nexo_symbol)
            btc_ticker = self.exchange.fetch_ticker(self.btc_symbol)
            
            nexo_price = nexo_ticker['last']
            btc_price = btc_ticker['last']
            
            logger.info(f"Current prices - NEXO: ${nexo_price:.4f}, BTC: ${btc_price:.2f}")
            return nexo_price, btc_price
        except Exception as e:
            logger.error(f"Error fetching prices: {e}")
            raise
    
    def fetch_historical_data(self, days_back: int = 90) -> pd.DataFrame:
        """
        Fetch historical price data for both assets.
        
        Args:
            days_back: Number of days of historical data to fetch
            
        Returns:
            DataFrame with historical price data
        """
        try:
            # Calculate start time
            since = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)
            
            # Fetch OHLCV data
            nexo_ohlcv = self.exchange.fetch_ohlcv(self.nexo_symbol, self.timeframe, since)
            btc_ohlcv = self.exchange.fetch_ohlcv(self.btc_symbol, self.timeframe, since)
            
            # Convert to DataFrames
            nexo_df = pd.DataFrame(nexo_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            btc_df = pd.DataFrame(btc_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            
            # Convert timestamps to datetime
            nexo_df['timestamp'] = pd.to_datetime(nexo_df['timestamp'], unit='ms')
            btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'], unit='ms')
            
            # Merge DataFrames
            merged_df = pd.merge(
                nexo_df[['timestamp', 'close']].rename(columns={'close': 'nexo_price'}),
                btc_df[['timestamp', 'close']].rename(columns={'close': 'btc_price'}),
                on='timestamp'
            )
            
            # Calculate ratio
            merged_df['nexo_btc_ratio'] = merged_df['nexo_price'] / merged_df['btc_price']
            
            logger.info(f"Fetched historical data: {len(merged_df)} entries")
            return merged_df
        except Exception as e:
            logger.error(f"Error fetching historical data: {e}")
            raise
    
    def update_historical_data(self) -> None:
        """Update historical data with latest values."""
        try:
            # Fetch historical data if we don't have enough
            if len(self.history_df) < self.window_size * 2:
                days_needed = max(90, self.window_size // 24 * 4)  # Ensure we get enough data
                new_data = self.fetch_historical_data(days_back=days_needed)
                self.history_df = new_data
            else:
                # Get current prices
                nexo_price, btc_price = self.fetch_current_prices()
                
                # Create new entry
                new_entry = pd.DataFrame([{
                    'timestamp': datetime.now(),
                    'nexo_price': nexo_price,
                    'btc_price': btc_price,
                    'nexo_btc_ratio': nexo_price / btc_price
                }])
                
                # Append to history
                self.history_df = pd.concat([self.history_df, new_entry], ignore_index=True)
            
            # Calculate analytics
            self._calculate_analytics()
            
            # Save updated data
            self._save_history()
        except Exception as e:
            logger.error(f"Error updating historical data: {e}")
            raise
    
    def _calculate_analytics(self) -> None:
        """Calculate analytics on the historical data."""
        df = self.history_df.copy()
        
        # Ensure data is sorted by timestamp
        df = df.sort_values('timestamp')
        
        # Calculate moving average
        df['ratio_sma'] = df['nexo_btc_ratio'].rolling(window=self.window_size, min_periods=1).mean()
        
        # Calculate percentile within recent window
        df['ratio_percentile'] = df['nexo_btc_ratio'].rolling(window=self.window_size, min_periods=1).apply(
            lambda x: np.percentile(np.nan_to_num(x), 100, interpolation='linear') 
            if np.isnan(x).all() else 
            percentileofscore(x.dropna(), x.iloc[-1])
        )
        
        # Identify peaks (local maxima within the window that exceed threshold)
        df['is_peak'] = False
        
        # Find local peaks within the window size
        for i in range(self.window_size, len(df)):
            window = df['nexo_btc_ratio'].iloc[i-self.window_size:i+1]
            if window.iloc[-1] > window.iloc[:-1].quantile(self.peak_threshold):
                df.at[df.index[i], 'is_peak'] = True
        
        self.history_df = df
    
    def _save_history(self) -> None:
        """Save historical data to CSV file."""
        history_file = os.path.join(self.data_dir, f"nexo_btc_history_{self.timeframe}.csv")
        self.history_df.to_csv(history_file, index=False)
        logger.info(f"Saved historical data to {history_file}")
        
        # For Google Colab: also make the file available for download
        try:
            files.download(history_file)
        except:
            pass  # Ignore if not in Colab environment
    
    def check_exchange_opportunity(self) -> Optional[Dict]:
        """Check if current market conditions present a good exchange opportunity."""
        if len(self.history_df) < self.window_size:
            logger.warning("Not enough historical data to evaluate opportunity")
            return None
        
        # Get latest data point
        latest = self.history_df.iloc[-1]
        
        # Calculate percentile in the recent window
        recent_window = self.history_df['nexo_btc_ratio'].iloc[-self.window_size:]
        current_percentile = percentileofscore(recent_window.dropna(), recent_window.iloc[-1])
        
        # Check if we're at or near a peak
        is_opportunity = False
        confidence = 0.0
        
        if latest['is_peak']:
            is_opportunity = True
            confidence = 0.9
        elif current_percentile > self.peak_threshold * 100:
            is_opportunity = True
            confidence = 0.7 + (current_percentile - self.peak_threshold * 100) / (100 - self.peak_threshold * 100) * 0.2
        
        if is_opportunity:
            return {
                'timestamp': latest['timestamp'],
                'nexo_price': latest['nexo_price'],
                'btc_price': latest['btc_price'],
                'nexo_btc_ratio': latest['nexo_btc_ratio'],
                'percentile': current_percentile,
                'confidence': confidence,
                'message': f"Good opportunity to exchange NEXO for BTC (confidence: {confidence:.2f})"
            }
        
        return None
    
    def plot_ratio_history(self, figsize=(12, 6)) -> None:
        """Plot the history of the NEXO/BTC ratio with peaks highlighted."""
        if len(self.history_df) < 2:
            logger.warning("Not enough data to generate plot")
            return
        
        plt.figure(figsize=figsize)
        
        # Plot ratio
        plt.plot(self.history_df['timestamp'], self.history_df['nexo_btc_ratio'], label='NEXO/BTC Ratio')
        
        # Plot moving average
        plt.plot(self.history_df['timestamp'], self.history_df['ratio_sma'], 
                 label=f'SMA ({self.window_size} periods)', linestyle='--')
        
        # Highlight peaks
        peaks = self.history_df[self.history_df['is_peak']]
        if not peaks.empty:
            plt.scatter(peaks['timestamp'], peaks['nexo_btc_ratio'], 
                        color='red', marker='^', s=100, label='Exchange Opportunity')
        
        # Formatting
        plt.title('NEXO/BTC Price Ratio History')
        plt.xlabel('Date')
        plt.ylabel('Price Ratio')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
    
    def display_opportunity_info(self, opportunity) -> None:
        """Display opportunity information in a nice format."""
        if not opportunity:
            display(HTML('<h3 style="color:orange;">No favorable exchange opportunity detected at this time</h3>'))
            return
        
        html = f"""
        <div style="background-color:#f8f9fa; padding:20px; border-radius:10px; border:1px solid #28a745;">
            <h3 style="color:#28a745;">{opportunity['message']}</h3>
            <ul>
                <li><b>Time:</b> {opportunity['timestamp']}</li>
                <li><b>NEXO Price:</b> ${opportunity['nexo_price']:.4f}</li>
                <li><b>BTC Price:</b> ${opportunity['btc_price']:.2f}</li>
                <li><b>NEXO/BTC Ratio:</b> {opportunity['nexo_btc_ratio']:.8f}</li>
                <li><b>Percentile:</b> {opportunity['percentile']:.2f}%</li>
                <li><b>Confidence:</b> {opportunity['confidence']:.2f}</li>
            </ul>
        </div>
        """
        display(HTML(html))

## Configuration and Setup

Now let's set up the parameters for our analysis:

In [ ]:
# Create data directory in Colab environment
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

# Configuration parameters
exchange_id = 'binance'  # Options: 'binance', 'kucoin', 'kraken', etc.
nexo_symbol = 'NEXO/USDT'  # Adjust if needed
btc_symbol = 'BTC/USDT'   # Adjust if needed
timeframe = '1h'         # Options: '5m', '15m', '1h', '4h', '1d'
window_size = 168        # 1 week of hourly data
peak_threshold = 0.9     # 90th percentile

# Initialize the monitor
monitor = NexoBTCExchangeMonitor(
    exchange_id=exchange_id,
    nexo_symbol=nexo_symbol,
    btc_symbol=btc_symbol,
    timeframe=timeframe,
    window_size=window_size,
    peak_threshold=peak_threshold,
    data_dir=data_dir
)

## Fetch Historical Data

First, let's fetch historical data and update our database:

In [ ]:
# Update historical data
monitor.update_historical_data()

# Display the first few rows of our data
monitor.history_df.head()

## Visualize the Ratio History

Let's visualize the NEXO/BTC ratio over time to see the historical pattern:

In [ ]:
# Plot the ratio history
monitor.plot_ratio_history(figsize=(14, 8))

## Check for Current Exchange Opportunities

Now let's check if there's currently a good opportunity to exchange NEXO for BTC:

In [ ]:
# Check for opportunity
opportunity = monitor.check_exchange_opportunity()

# Display results
monitor.display_opportunity_info(opportunity)

## Interactive Analysis Tools

Let's create some interactive tools to analyze the data in more detail:

In [ ]:
# View data statistics
print("NEXO/BTC Ratio Statistics:")
ratio_stats = monitor.history_df['nexo_btc_ratio'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
display(ratio_stats)

# Plot histogram of the ratio
plt.figure(figsize=(10, 6))
plt.hist(monitor.history_df['nexo_btc_ratio'], bins=50, alpha=0.7)
plt.axvline(monitor.history_df['nexo_btc_ratio'].iloc[-1], color='red', linestyle='--', 
            label=f'Current Ratio: {monitor.history_df["nexo_btc_ratio"].iloc[-1]:.8f}')
plt.title('Distribution of NEXO/BTC Ratio')
plt.xlabel('Ratio Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## Trend Analysis

Let's look at the recent trend to see if we're approaching a peak:

In [ ]:
# Get the recent window
recent_data = monitor.history_df.tail(monitor.window_size).copy()

# Add a column to show days from now
recent_data['days_ago'] = (datetime.now() - recent_data['timestamp']).dt.total_seconds() / (60*60*24)

# Plot recent trend
plt.figure(figsize=(14, 8))

# Plot ratio
plt.plot(recent_data['timestamp'], recent_data['nexo_btc_ratio'], label='NEXO/BTC Ratio')

# Add percentile threshold line
threshold_value = recent_data['nexo_btc_ratio'].quantile(monitor.peak_threshold)
plt.axhline(threshold_value, color='green', linestyle='--', 
            label=f'{monitor.peak_threshold*100}th Percentile: {threshold_value:.8f}')

# Highlight peaks
peaks = recent_data[recent_data['is_peak']]
if not peaks.empty:
    plt.scatter(peaks['timestamp'], peaks['nexo_btc_ratio'], 
                color='red', marker='^', s=100, label='Exchange Opportunity')

# Show current ratio
current_ratio = recent_data['nexo_btc_ratio'].iloc[-1]
plt.scatter(recent_data['timestamp'].iloc[-1], current_ratio, 
            color='blue', marker='o', s=100, label=f'Current: {current_ratio:.8f}')

# Formatting
plt.title('Recent NEXO/BTC Price Ratio Trend')
plt.xlabel('Date')
plt.ylabel('Price Ratio')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## Continuous Monitoring

If you want to continuously monitor for opportunities, you can run the following cell. This will check for opportunities every hour (or at your specified interval):

In [ ]:
# Run this cell if you want to continuously monitor
# Note: In Colab, this will run until you interrupt it or the session times out

interval_minutes = 60  # Check every hour
max_iterations = 24    # Run for 24 hours max

try:
    for iteration in range(1, max_iterations + 1):
        print(f"\n\n{'='*80}\nMonitoring iteration {iteration}/{max_iterations} at {datetime.now()}\n{'='*80}")
        
        # Update data
        monitor.update_historical_data()
        
        # Check for opportunity
        opportunity = monitor.check_exchange_opportunity()
        
        # Display results
        monitor.display_opportunity_info(opportunity)
        
        # Generate plot
        monitor.plot_ratio_history()
        
        # If this is not the last iteration, sleep
        if iteration < max_iterations:
            print(f"Sleeping for {interval_minutes} minutes until next check...")
            time.sleep(interval_minutes * 60)
        
except KeyboardInterrupt:
    print("Monitoring stopped by user")
except Exception as e:
    print(f"Error in monitoring loop: {e}")

## Download Historical Data

You can download the historical data collected during this session:

In [ ]:
# Save and download the data
history_file = os.path.join(data_dir, f"nexo_btc_history_{monitor.timeframe}.csv")
monitor.history_df.to_csv(history_file, index=False)
print(f"Saved data to {history_file}")

try:
    files.download(history_file)
    print("Download initiated. Check your browser's download folder.")
except:
    print("Not running in Google Colab or download failed.")

## Customize Parameters

You can re-run the analysis with different parameters using the cell below:

In [ ]:
# Change these parameters to customize your analysis
new_timeframe = '4h'    # Try different timeframes: '5m', '15m', '1h', '4h', '1d'
new_window = 42         # 42 periods of 4h = 1 week
new_threshold = 0.85    # 85th percentile

# Initialize new monitor with custom parameters
custom_monitor = NexoBTCExchangeMonitor(
    exchange_id=exchange_id,
    nexo_symbol=nexo_symbol,
    btc_symbol=btc_symbol,
    timeframe=new_timeframe,
    window_size=new_window,
    peak_threshold=new_threshold,
    data_dir=data_dir
)

# Update data and check for opportunities
custom_monitor.update_historical_data()
opportunity = custom_monitor.check_exchange_opportunity()

# Display results
print(f"Analysis with timeframe={new_timeframe}, window={new_window}, threshold={new_threshold}:")
custom_monitor.display_opportunity_info(opportunity)
custom_monitor.plot_ratio_history()